
<img src="images/logos/TClogo.png">

### presents

# ROS Developers Live Class n54

# ROS Control Intro: Build your own controller

This notebook is additional material for the **ROS Developers Live Class n.54** created and provided for free by **Alberto Ezquerro** and **Ricardo Tellez** of <a href="www.theconstructsim.com">The Construct</a>. You can distribute this notebook as long as you provide copy of this paragraph with it.

## Why this class?

Up to this point, you are using the default controllers that the ros_control packages provide, which is the most common practice. In some cases, it may be interesting to create a custom ROS controller that sends specific commands to your joints.

In today's live class, we are going to learn the following contents:
1. **How to create a ROS Controller for joints**
2. **How to use it in your robot**

Pre-requisites for this live class are:
* Basic knowledge of <b>ROS concepts such as topics, publish and subscribe, ROS Service</b> using **C++**. If you don't know about it, <a href="http://www.theconstructsim.com/construct-learn-develop-robots-using-ros/robotigniteacademy_learnros/ros-courses-library/ros-courses-ros-basics-in-5-days-c/">check this course</a>
* Knowledge of C++
* Knowledge of <b>URDF format for robot definition</b>. If you don't know about it, <a href="http://www.theconstructsim.com/construct-learn-develop-robots-using-ros/robotigniteacademy_learnros/ros-courses-library/robot-creation-with-urdf/">check this course</a>.
* Love for Robotics 
* ...that's it!!

## Simulation to be used today

We are going to use the Pi Robot simulation

<img src="images/pirobot.png">

### How to use this ROSject

A <a href="http://rosjects.com">**ROSject**</a> is a **ROS project** packaged in such a way that all the material it contains (**ROS code, Gazebo simulations and Notebooks**) can be shared with any body **using only a web link**. That is what we did with all the attendants to the Live Class, we shared this ROSject with them (so they can have access to all the ROS material they contain).

**Check <a heref="https://www.youtube.com/watch?v=cR-Ow5K7oSo">this webinar</a> to learn more about ROSjects and how to create your own ROSjects**.

You will need to have a free account at the <a href="http://rosds.online">ROS Development Studio</a> (ROSDS). Get the account and then follow the indications below.

## 0. Requirements for the controller

* It will be a controller to control effort based joints
* It is going to accept position values
* It is going to be called **MyPositionController**

## 1. Creating the package

Create a new package named my_controller with the following dependencies: <i><b>roscpp</b></i>, <i><b>pluginlib</b></i>, <i><b>controller_interface</b></i>, and <i><b>hardware_interface</b></i>. To do that, just execute the following command:

In [ ]:
$ cd ~/catkin_ws/src
$ catkin_create_pkg my_controller roscpp pluginlib controller_interface hardware_interface

<img src="images/ROS-Control-overview.png">

## 2. Creating the source code of the controller

Inside the src folder that was created in your new package, create a file named <i><b>my_controller.cpp</b></i>. Let's populate it!

#### Add the required include files

Include the libraries that allow us to define a new controller, a new hardware interface and a new plugin

In [ ]:
#include <controller_interface/controller.h>
#include <hardware_interface/joint_command_interface.h>
#include <pluginlib/class_list_macros.h>
#include <std_msgs/Float64.h>


#### Add a name space for our controller

This namespace allow us to differentiate controllers that have the same name, but different implementation (so we do not confuse ourselves):

In [ ]:
namespace my_controller_ns
{
    
    
}

#### Create the class of the controller

Since we are creating an effort controller we are going to create our controller inheriting from the hardware interface **EffortJointInterface**. This means that our controller has to follow that interface in order to be able to properly communicate with an effort joint in the hardware.

In [ ]:
class MyPositionController : public controller_interface::Controller<hardware_interface::EffortJointInterface>
{
    
};

#### Initialize the controller

The *init()* function is called when your controller is loaded by the *controller manager*. Inside this function, you must:

* get the **name of the joint** that you will control from the Parameter Server first (so from the YAML file, which you will modify later)
* get the actual **joint object** that provides access to the resources of the joint. We will use that object in the **realtime loop**.
* get the value of the **gain** parameter
* start the subscriber for receiving **desired joint positions**

In [ ]:
        bool init(hardware_interface::EffortJointInterface* hw, ros::NodeHandle &n)
        {
            std::string my_joint;
            if (!n.getParam("joint", my_joint))
            {
                ROS_ERROR("Could not find joint name");
                return false;
            }
            joint_ = hw->getHandle(my_joint);  // throws on failure
            command_ = joint_.getPosition();   // set the current joint goal to the current joint position

            // Load gain using gains set on parameter server
            if (!n.getParam("gain", gain_))
            {
                ROS_ERROR("Could not find the gain parameter value");
                return false;
            }

            // Start command subscriber
            sub_command_ = n.subscribe<std_msgs::Float64>("command", 1, &MyPositionController::setCommandCB, this);

            return true;
        }


#### Create the *update* loop 

The update loop is **the control loop that is going to be executed by the controller**. It is here where you put your controller code.

Here you are defining the command that you are going to send to your joint. In this case, it's a product between an * **error** * variable and a * **gain_** * variable. The error variable is defined as the difference between the current position (* **joint_.getPosition()** *) of the joint and the goal position (* **setpoint_** *) of the joint.

In this example we have put a very simple controller. You can create the controller as complex as possible, but remember not to put too much computation to prevent to break the controller loop time.

In [ ]:
        void update(const ros::Time& time, const ros::Duration& period)
        {
            double error = command_ - joint_.getPosition();
            double commanded_effort = error*gain_;
            joint_.setCommand(commanded_effort);
        }

        void setCommandCB(const std_msgs::Float64ConstPtr& msg)
        {
            command_ = msg->data;
        }

#### Add the *start* and *stop* functions

You need to add two functions that will be executed during the start and stop of the controller by the controller manager. 

Put in those the content you want to be done for **initialization of the controller** and for **stopping of the controller**.

In [ ]:
void starting(const ros::Time& time) { }
void stopping(const ros::Time& time) { }

#### Add some required class members

Those are required to make the class work

In [ ]:
        private:
            hardware_interface::JointHandle joint_;
            double gain_;
            double command_;
            ros::Subscriber sub_command_;

#### Register this library as a controller available

Here, you are calling the special macro plugin <i><b>PLUGINLIB_EXPORT_CLASS</b></i> in order to allow this class to be dynamically loaded.

In [ ]:
PLUGINLIB_EXPORT_CLASS(my_controller_ns::MyPositionController, controller_interface::ControllerBase);

More info about how the pluginlib works, <a href="http://wiki.ros.org/pluginlib">here</a>.

#### The full controller code

At the end, the code of your controller should look like this: 

In [ ]:
#include <controller_interface/controller.h>
#include <hardware_interface/joint_command_interface.h>
#include <pluginlib/class_list_macros.h>
#include <std_msgs/Float64.h>

namespace my_controller_ns
{
    class MyPositionController : public controller_interface::Controller<hardware_interface::EffortJointInterface>
    {

        bool init(hardware_interface::EffortJointInterface* hw, ros::NodeHandle &n)
        {
            std::string my_joint;
            if (!n.getParam("joint", my_joint))
            {
                ROS_ERROR("Could not find joint name");
                return false;
            }
            joint_ = hw->getHandle(my_joint);  // throws on failure
            command_ = joint_.getPosition();

            // Load gain using gains set on parameter server
            if (!n.getParam("gain", gain_))
            {
                ROS_ERROR("Could not find the gain parameter value");
                return false;
            }

            // Start command subscriber
            sub_command_ = n.subscribe<std_msgs::Float64>("command", 1, &MyPositionController::setCommandCB, this);

            return true;
        }

        void update(const ros::Time& time, const ros::Duration& period)
        {
            double error = command_ - joint_.getPosition();
            double commanded_effort = error*gain_;
            joint_.setCommand(commanded_effort);
        }

        void setCommandCB(const std_msgs::Float64ConstPtr& msg)
        {
            command_ = msg->data;
        }

        void starting(const ros::Time& time) { }
        void stopping(const ros::Time& time) { }

        private:
            hardware_interface::JointHandle joint_;
            double gain_;
            double command_;
            ros::Subscriber sub_command_;
    };

    PLUGINLIB_EXPORT_CLASS(my_controller_ns::MyPositionController, controller_interface::ControllerBase);
}

## 3. Compiling the controller

#### Create a plugin description file

Inside your package, create a new file named **controller_plugins.xml**. This file is a description of the controller plugin. It indicates the path where the library will be placed when we compile the controller, as well as the name, type, and class of the controller. 

Copy the following code into it:

In [ ]:
<library path="lib/libmy_controller_lib">
  <class name="my_controller/MyPositionController" 
         type="my_controller_ns::MyPositionController"           
         base_class_type="controller_interface::ControllerBase" />
</library>

#### Update the *package.xml* file

This line indicates that our package will provide a plugin. This is very important because if we don't add this line, the controller manager won't be able to find our new controller and load it.

Go to the <i><b>package.xml</b></i> file of your package. Inside the <i><b>&lt;export&gt;</b></i> tag, place the following line:

In [ ]:
<controller_interface plugin="${prefix}/controller_plugins.xml"/>

#### Update the *CMakeLists.txt* file

Go to the CMakeLists.txt file of your package. Find the add_library() and target_link_libraries() functions, and replace them with these:

In [ ]:
add_library(my_controller_lib src/my_controller.cpp)

In [ ]:
target_link_libraries(my_controller_lib ${catkin_LIBRARIES})

#### Build the controller

Go to the catkin_ws directory and compile your package

In [ ]:
$ catkin_make

Check that the controller has been properly registered:

In [ ]:
$ rospack plugins --attrib=plugin controller_interface | grep my_controller

You should get a response like this:

In [ ]:
my_controller /home/user/catkin_ws/src/my_controller/controller_plugins.xml

## 4. Launching the controller

Let's change the controller of the **head_pan_joint** to use our new controller.

#### Modify the configuration file

Go to the controllers config file and modify the controller for the *head_pan_joint* to use our newly created one.

In [ ]:
  head_pan_joint_position_controller:
    type: my_controller/MyPositionController
    joint: head_pan_joint
    gain: 0.1

#### Let's launch it!

Launch it by going to **Simulations->Select Launch file->main.launch**. You can also launch it from a shell, by calling the launch file and the opening the Gazebo window (on **Tools->Gazebo**).

<img src="images/pirobot2.png">

Launch the controllers:

In [ ]:
$ roslaunch pi_robot_pkg controllers.launch

Check that the controller has been loaded properly:

In [ ]:
$ rosservice call /pi_robot/controller_manager/list_controllers "{}"

Check the the joint works properly by sending to it a goal:

In [ ]:
$ rostopic pub /pi_robot/head_pan_joint_position_controller/command std_msgs/Float64 "data: 1.0"

# Mission completed!!

## Homework

* Add a neural controller for the *left_shoulder_forward_joint* joint of the Pi Robot and be able to make it move with it.
    * the neural network takes as input the desired position and the current position
    * takes as output the effort command to send to the joint
    * You can use Tensorflow to build and train the network (here, a <a href="http://www.theconstructsim.com/construct-learn-develop-robots-using-ros/robotigniteacademy_learnros/ros-courses-library/ros-deep-learning-with-tensor-flow-101-python/">**course on how to use Tensorflow with ROS**</a>).
    * You can generate the training data from a PID controller
* Send me your questions and results at *rtellez@theconstructsim.com*

...

### Before you log off, remember to <span style="background: #098be8; padding: 10px; color:white;">GIVE US A LIKE</span> and hit the <span style="background: #098be8; padding: 10px; color:white;">THUMBS UP</span> and <span style="background: #098be8; padding: 10px; color:white;">SUBSCRIBE</span> for more weekly tutorials!!!

## Don't miss the most practical online ROS Conference of 2019!!

<img src="images/rosdevcon2.png">

A **hands on conference** where you will **learn and practice** at the same time **with the speakers**. 

It is an online conference with the same format of the Live Classes. You can attend from anywhere and will get a **rosject** with all the content of the speakers.

#### 8 speakers - 8 practical ROS projects on a single weekend

### You can also be a speaker of the conference. Check the call for papers

### More information here: <a href="http://www.rosdevcon.com">www.rosdevcon.com</a>

### Check the videos of the previous ROSDevCon 2018 <a href="">here</a>

 <a href="https://goo.gl/7ApVAp"><img src="images/logos/RIAlogo.png"></a>

We have an online academy that teaches you more about how to control robots with ROS. 

### Check the following related courses with even deeper explanations:

* <a href="http://www.theconstructsim.com/construct-learn-develop-robots-using-ros/robotigniteacademy_learnros/ros-courses-library/ros-control-101/">ROS Control 101</a>

<img src="images/control.png">

* <a href="http://www.theconstructsim.com/construct-learn-develop-robots-using-ros/robotigniteacademy_learnros/ros-courses-library/robot-creation-with-urdf/">ROS URDF Robot Description</a>

<img src="images/urdf.png">


# KEEP PUSHING YOUR ROS LEARNING!